In [ ]:
# 구글 코랩에서 사용한 스켈레탈 코드

from pycocotools.coco import COCO
import numpy as np 
import skimage.io as io 
import matplotlib.pyplot as plt 
import pylab
import json

NthPerson = '001'
NthFrame = '0121'

# imgPath = f'/content/drive/MyDrive/golf/20201210_General_114_DOC_A_M30_MM_{NthPerson}_{NthFrame}.jpg'
# annPath = f'/content/drive/MyDrive/Colab Notebooks/HRD/golf_coco/20200612_General_003_DOS_P_F20_MM/20200612_General_003_DOS_P_F20_MM_{NthPerson}_{NthFrame}.json'

In [ ]:
import torchvision
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
# create a model object from the keypointrcnn_resnet50_fpn class
model = torchvision.models.detection.keypointrcnn_resnet50_fpn(pretrained=True)
# call the eval() method to prepare the model for inference mode.
model.eval()

In [ ]:
# create the list of keypoints.
keypoints = ['nose','left_eye','right_eye','left_ear','right_ear','left_shoulder','right_shoulder','left_elbow',
                'right_elbow','left_wrist','right_wrist','left_hip','right_hip','left_knee', 'right_knee', 'left_ankle','right_ankle']

In [ ]:
# get the limbs
def get_limbs_from_keypoints(keypoints):
    limbs = [       
        [keypoints.index('right_eye'), keypoints.index('nose')],
        [keypoints.index('right_eye'), keypoints.index('right_ear')],
        [keypoints.index('left_eye'), keypoints.index('nose')],
        [keypoints.index('left_eye'), keypoints.index('left_ear')],
        [keypoints.index('right_shoulder'), keypoints.index('right_elbow')],
        [keypoints.index('right_elbow'), keypoints.index('right_wrist')],
        [keypoints.index('left_shoulder'), keypoints.index('left_elbow')],
        [keypoints.index('left_elbow'), keypoints.index('left_wrist')],
        [keypoints.index('right_hip'), keypoints.index('right_knee')],
        [keypoints.index('right_knee'), keypoints.index('right_ankle')],
        [keypoints.index('left_hip'), keypoints.index('left_knee')],
        [keypoints.index('left_knee'), keypoints.index('left_ankle')],
        [keypoints.index('right_shoulder'), keypoints.index('left_shoulder')],
        [keypoints.index('right_hip'), keypoints.index('left_hip')],
        [keypoints.index('right_shoulder'), keypoints.index('right_hip')],
        [keypoints.index('left_shoulder'), keypoints.index('left_hip')]
        ]
    return limbs


limbs = get_limbs_from_keypoints(keypoints)

In [ ]:
from torchvision import transforms as T

# Read the image using opencv 
img = cv2.imread(imgPath)

# preprocess the input image
transform = T.Compose([T.ToTensor()])
img_tensor = transform(img)

# forward-pass the model
# the input is a list, hence the output will also be a list
output = model([img_tensor])[0]

print(output.keys())

In [ ]:
output["keypoints"]

In [ ]:
for i in range(10):
  y= output["keypoints"][0, :2]
  print(y)

In [ ]:
y= output["keypoints"][0, :2]
y

In [ ]:
torch.max(output["keypoints"]), torch.min(output["keypoints"])

In [ ]:

keypoints_img = draw_keypoints_per_person(img, output["keypoints"], output["keypoints_scores"], output["scores"],keypoint_threshold=2)

cv2.imwrite("output/keypoints-img.jpg", keypoints_img)

plt.figure(figsize=(15, 15))
plt.imshow(keypoints_img[:, :, ::-1])
plt.show()

In [ ]:
def draw_keypoints_per_person(img, all_keypoints, all_scores, confs, keypoint_threshold=2, conf_threshold=0.9):
    # initialize a set of colors from the rainbow spectrum
    cmap = plt.get_cmap('rainbow')
    # create a copy of the image
    img_copy = img.copy()
    # pick a set of N color-ids from the spectrum
    color_id = np.arange(1,255, 255//len(all_keypoints)).tolist()[::-1]
    # iterate for every person detected
    for person_id in range(len(all_keypoints)):
      # check the confidence score of the detected person
      if confs[person_id]>conf_threshold:
        # grab the keypoint-locations for the detected person
        keypoints = all_keypoints[person_id, ...]
        # grab the keypoint-scores for the keypoints
        scores = all_scores[person_id, ...]
        # iterate for every keypoint-score
        for kp in range(len(scores)):
            # check the confidence score of detected keypoint
            if scores[kp]>keypoint_threshold:
                # convert the keypoint float-array to a python-list of intergers
                keypoint = tuple(map(int, keypoints[kp, :2].detach().numpy().tolist()))
                # pick the color at the specific color-id
                color = tuple(np.asarray(cmap(color_id[person_id])[:-1])*255)
                # draw a cirle over the keypoint location
                cv2.circle(img_copy, keypoint, 10, color, -1)

    return img_copy

In [ ]:
# overlay the skeleton in the detected person
skeletal_img = draw_skeleton_per_person(img, output["keypoints"], output["keypoints_scores"], output["scores"],keypoint_threshold=2)

In [ ]:
plt.figure(figsize=(15, 15))
plt.imshow(skeletal_img)
plt.show()